In [1]:
import os
from bs4 import BeautifulSoup #parses the html and extract the pieces we want
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeOut
# playwright allows us to open HTML and grab it from its pages
import time

In [25]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [2]:
!playwright install

In [3]:
SEASONS = list(range(2016, 2025))
SEASONS

[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [4]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [9]:
#async function to handle playwright and
# allows the script to send multiple requests without waiting for each response before firing the next one
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        #in order to not scrape too quickly or else website bans us
        #first time goes through pauses for 5 seconds, 2: 10 seconds, 3: 15, etc
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page() # make new tab
                await page.goto(url) #send browser in the tab to the url
                print(await page.title())
                html = await page.inner_html(selector) #in order to not get all of the stuff on the page we pass
                #in a selector.
        except PlaywrightTimeOut:
            print(f"Timeout error on {url}")
            continue #if scraping fails loop through again
        else:
            break #if try block runs successfully
            
    return html

In [10]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    #PARSING WITH BEAUTIFUL SOUP
    soup = BeautifulSoup(html)
    links = soup.find_all("a") #find all anchor tags
    href = [l["href"] for l in links]
    #href now provides the individual links to each month's standings of that season
    
    #must convert the "link" into a complete url
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
            
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

    

In [11]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
Timeout error on https://basketball-reference.com/leagues/NBA_2017_games-october.html
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
Timeout error on https://basketball-reference.com/leagues/NBA_2017_games-may.html
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
Timeout error on https://basketball-reference.com/leagues/NBA_2018_games-december.html
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Ba

# Parsing box score links with beautiful soup

In [13]:
standings_files = os.listdir(STANDINGS_DIR)

In [22]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    #need to filter the links to be just the boxscores
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    #add host name
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [26]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
        
    await scrape_game(filepath)

Mavericks vs Raptors, October 23, 2021 | Basketball-Reference.com
Wizards vs Nets, October 25, 2021 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202110250MIN.html
Pelicans vs Timberwolves, October 25, 2021 | Basketball-Reference.com
Cavaliers vs Nuggets, October 25, 2021 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202110290BRK.html
Pacers vs Nets, October 29, 2021 | Basketball-Reference.com
Celtics vs Wizards, October 30, 2021 | Basketball-Reference.com
Magic vs Pistons, October 30, 2021 | Basketball-Reference.com
Raptors vs Pacers, October 30, 2021 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202106100UTA.html
2021 NBA Western Conference Semifinals Game 2: Clippers vs Jazz, June 10, 2021 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202106110DEN.html
2021 NBA Western Conference Semifinals Game 3: Suns vs

Bucks vs Pacers, January 27, 2023 | Basketball-Reference.com
Grizzlies vs Timberwolves, January 27, 2023 | Basketball-Reference.com
Magic vs Heat, January 27, 2023 | Basketball-Reference.com
Cavaliers vs Thunder, January 27, 2023 | Basketball-Reference.com
Raptors vs Warriors, January 27, 2023 | Basketball-Reference.com
Nuggets vs 76ers, January 28, 2023 | Basketball-Reference.com
Knicks vs Nets, January 28, 2023 | Basketball-Reference.com
Rockets vs Pistons, January 28, 2023 | Basketball-Reference.com
Bulls vs Magic, January 28, 2023 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202301280ATL.html
Timeout error on https://www.basketball-reference.com/boxscores/202301280ATL.html
Clippers vs Hawks, January 28, 2023 | Basketball-Reference.com
Kings vs Timberwolves, January 28, 2023 | Basketball-Reference.com
Wizards vs Pelicans, January 28, 2023 | Basketball-Reference.com
Suns vs Spurs, January 28, 2023 | Basketball-Reference.com
Lakers vs Celt

Heat vs Pistons, April 12, 2016 | Basketball-Reference.com
76ers vs Raptors, April 12, 2016 | Basketball-Reference.com
Thunder vs Spurs, April 12, 2016 | Basketball-Reference.com
Grizzlies vs Clippers, April 12, 2016 | Basketball-Reference.com
Heat vs Celtics, April 13, 2016 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/201604130BRK.html
Raptors vs Nets, April 13, 2016 | Basketball-Reference.com
76ers vs Bulls, April 13, 2016 | Basketball-Reference.com
Magic vs Hornets, April 13, 2016 | Basketball-Reference.com
Pistons vs Cavaliers, April 13, 2016 | Basketball-Reference.com
Spurs vs Mavericks, April 13, 2016 | Basketball-Reference.com
Kings vs Rockets, April 13, 2016 | Basketball-Reference.com
Pacers vs Bucks, April 13, 2016 | Basketball-Reference.com
Pelicans vs Timberwolves, April 13, 2016 | Basketball-Reference.com
Hawks vs Wizards, April 13, 2016 | Basketball-Reference.com
Grizzlies vs Warriors, April 13, 2016 | Basketball-Reference.com


Grizzlies vs Nets, March 4, 2024 | Basketball-Reference.com
Clippers vs Bucks, March 4, 2024 | Basketball-Reference.com
Trail Blazers vs Timberwolves, March 4, 2024 | Basketball-Reference.com
Wizards vs Jazz, March 4, 2024 | Basketball-Reference.com
Bulls vs Kings, March 4, 2024 | Basketball-Reference.com
Thunder vs Lakers, March 4, 2024 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202403050CHO.html
Timeout error on https://www.basketball-reference.com/boxscores/202403050CHO.html
Timeout error on https://www.basketball-reference.com/boxscores/202403050CHO.html
76ers vs Nets, March 5, 2024 | Basketball-Reference.com
Celtics vs Cavaliers, March 5, 2024 | Basketball-Reference.com
Pistons vs Heat, March 5, 2024 | Basketball-Reference.com
Hawks vs Knicks, March 5, 2024 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202403050TOR.html
Pelicans vs Raptors, March 5, 2024 | Basketball-Reference.com
Spurs vs

Heat vs Cavaliers, March 20, 2024 | Basketball-Reference.com
Pacers vs Pistons, March 20, 2024 | Basketball-Reference.com
Bucks vs Celtics, March 20, 2024 | Basketball-Reference.com
Kings vs Raptors, March 20, 2024 | Basketball-Reference.com
Jazz vs Thunder, March 20, 2024 | Basketball-Reference.com
Grizzlies vs Warriors, March 20, 2024 | Basketball-Reference.com
76ers vs Suns, March 20, 2024 | Basketball-Reference.com
Clippers vs Trail Blazers, March 20, 2024 | Basketball-Reference.com
Pelicans vs Magic, March 21, 2024 | Basketball-Reference.com
Kings vs Wizards, March 21, 2024 | Basketball-Reference.com
Bulls vs Rockets, March 21, 2024 | Basketball-Reference.com
Nets vs Bucks, March 21, 2024 | Basketball-Reference.com
Jazz vs Mavericks, March 21, 2024 | Basketball-Reference.com
Knicks vs Nuggets, March 21, 2024 | Basketball-Reference.com
Hawks vs Suns, March 21, 2024 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202403220DET.html
Celtics v

...

2021 NBA Western Conference First Round Game 2: Trail Blazers vs Nuggets, May 24, 2021 | Basketball-Reference.com
2021 NBA Eastern Conference First Round Game 2: Celtics vs Nets, May 25, 2021 | Basketball-Reference.com
2021 NBA Western Conference First Round Game 2: Lakers vs Suns, May 25, 2021 | Basketball-Reference.com
2021 NBA Western Conference First Round Game 2: Mavericks vs Clippers, May 25, 2021 | Basketball-Reference.com
2021 NBA Eastern Conference First Round Game 2: Wizards vs 76ers, May 26, 2021 | Basketball-Reference.com
2021 NBA Eastern Conference First Round Game 2: Hawks vs Knicks, May 26, 2021 | Basketball-Reference.com
2021 NBA Western Conference First Round Game 2: Grizzlies vs Jazz, May 26, 2021 | Basketball-Reference.com
2021 NBA Eastern Conference First Round Game 3: Bucks vs Heat, May 27, 2021 | Basketball-Reference.com
2021 NBA Western Conference First Round Game 3: Suns vs Lakers, May 27, 2021 | Basketball-Reference.com
2021 NBA Western Conference First Round G

Pelicans vs Lakers, April 1, 2022 | Basketball-Reference.com
Hornets vs 76ers, April 2, 2022 | Basketball-Reference.com
Cavaliers vs Knicks, April 2, 2022 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202204020ATL.html
Nets vs Hawks, April 2, 2022 | Basketball-Reference.com
Heat vs Bulls, April 2, 2022 | Basketball-Reference.com
Jazz vs Warriors, April 2, 2022 | Basketball-Reference.com
Wizards vs Celtics, April 3, 2022 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202204030MIL.html
Timeout error on https://www.basketball-reference.com/boxscores/202204030MIL.html
Mavericks vs Bucks, April 3, 2022 | Basketball-Reference.com
Nuggets vs Lakers, April 3, 2022 | Basketball-Reference.com
Pistons vs Pacers, April 3, 2022 | Basketball-Reference.com
76ers vs Cavaliers, April 3, 2022 | Basketball-Reference.com
Knicks vs Magic, April 3, 2022 | Basketball-Reference.com
Timeout error on https://www.basketball-

2022 NBA Eastern Conference First Round Game 3: Bucks vs Bulls, April 22, 2022 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202204220NOP.html
Timeout error on https://www.basketball-reference.com/boxscores/202204220NOP.html
2022 NBA Western Conference First Round Game 3: Suns vs Pelicans, April 22, 2022 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202204230TOR.html
2022 NBA Eastern Conference First Round Game 4: 76ers vs Raptors, April 23, 2022 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202204230UTA.html
2022 NBA Western Conference First Round Game 4: Mavericks vs Jazz, April 23, 2022 | Basketball-Reference.com
2022 NBA Eastern Conference First Round Game 3: Celtics vs Nets, April 23, 2022 | Basketball-Reference.com
2022 NBA Western Conference First Round Game 4: Grizzlies vs Timberwolves, April 23, 2022 | Basketball-Reference.com
Timeout error on 

Trail Blazers vs Warriors, December 8, 2021 | Basketball-Reference.com
Magic vs Kings, December 8, 2021 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202112080LAC.html
Timeout error on https://www.basketball-reference.com/boxscores/202112080LAC.html
Celtics vs Clippers, December 8, 2021 | Basketball-Reference.com
Jazz vs 76ers, December 9, 2021 | Basketball-Reference.com
Lakers vs Grizzlies, December 9, 2021 | Basketball-Reference.com
Nuggets vs Spurs, December 9, 2021 | Basketball-Reference.com
Kings vs Hornets, December 10, 2021 | Basketball-Reference.com
Mavericks vs Pacers, December 10, 2021 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202112100ATL.html
Nets vs Hawks, December 10, 2021 | Basketball-Reference.com
Knicks vs Raptors, December 10, 2021 | Basketball-Reference.com
Bucks vs Rockets, December 10, 2021 | Basketball-Reference.com
Cavaliers vs Timberwolves, December 10, 2021 | Basketbal

Error: Page.goto: net::ERR_INTERNET_DISCONNECTED at https://www.basketball-reference.com/boxscores/202112110PHI.html
Call log:
navigating to "https://www.basketball-reference.com/boxscores/202112110PHI.html", waiting until "load"
